# San Francisco Bike Share
## Content
- Moduleimport
- Connection to Database
- Stations
    - Total count
    - Station Usage/Utilization - Start
    - Station Usage/Utilization - End
- Bicycles
    - Number of Bicycles
    - Total Bicycles (grouped by Cities)
- Rental Duration
    - Average Rental Duration
    - Average Rental Duration (grouped by Cities)
- Rental Behavior
    - Rental Behavior Difference by Weekday/Weekend
    - Rental Behavior Variations by Hour of the Day
- Customer Segmentation
    - Rental Behavior Difference between Subscribers and Casual Customers
- Total Trips

## Moduleimport

In [81]:
import psycopg2
from configparser import ConfigParser
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## Connection to Database

In [82]:
config = ConfigParser()
config.read("20231219_07_Anderes_config.ini", encoding="utf-8")

conn = psycopg2.connect(
    host = "localhost",
    port = "5432",
    user = "postgres",
    password = config["local_postgres"]["passwort"],
    database = "leihräder"
    )

## Stations
### Total Count

In [83]:
query1 = "SELECT COUNT(*) FROM station;"
df1 = pd.read_sql(query1, conn)
df1.set_index("count", inplace=True)
df1.index.name = "count"  
df1

""
count
70


### Station Usage/Utilization - Start

In [84]:
query2 = "SELECT start_station_name, COUNT(*) as usage_count FROM trip GROUP BY start_station_name ORDER BY usage_count DESC LIMIT 5;"
df2 = pd.read_sql(query2, conn)
df2.set_index("start_station_name", inplace=True)
df2.index.name = "start_station_name"  
df2

,usage_count
start_station_name,
San Francisco Caltrain (Townsend at 4th),28338
Harry Bridges Plaza (Ferry Building),16849
Market at Sansome,14953
Temporary Transbay Terminal (Howard at Beale),14207
Embarcadero at Sansome,13504


### Station Usage/Utilization - End

In [85]:
query3 = "SELECT end_station_name, COUNT(*) as usage_count FROM trip GROUP BY end_station_name ORDER BY usage_count DESC LIMIT 5;"
df3 = pd.read_sql(query3, conn)
df3.set_index("end_station_name", inplace=True)
df3.index.name = "end_station_name"  
df3

,usage_count
end_station_name,
San Francisco Caltrain (Townsend at 4th),32336
Market at Sansome,16837
Harry Bridges Plaza (Ferry Building),15822
Embarcadero at Sansome,15072
2nd at Townsend,12981


## Bicycles
### Number of Bicycles

In [86]:
query4 = "SELECT COUNT(DISTINCT bike_id) FROM trip;"
df4 = pd.read_sql(query4, conn)
df4.set_index("count", inplace=True)
df4.index.name = "count"  
df4

""
count
689


### Total Bicycles (grouped by Cities)

In [87]:
query5 = "SELECT s.city, COUNT(DISTINCT t.bike_id) AS number_of_bikes FROM trip t JOIN station s ON t.start_station_id = s.id GROUP BY s.city;"
df5 = pd.read_sql(query5, conn)
df5.set_index("city", inplace=True)
df5.index.name = "city"  
df5

,number_of_bikes
city,
Mountain View,199
Palo Alto,187
Redwood City,140
San Francisco,365
San Jose,280


## Rental Duration
### Average Rental Duration

In [88]:
query6 = "SELECT FLOOR((AVG(duration) % 3600) / 60) AS average_minutes FROM trip;"
df6 = pd.read_sql(query6, conn)
df6.set_index("average_minutes", inplace=True)
df6.index.name = "average_minutes"  
df6

""
average_minutes
17.0


### Average Rental Duration (grouped by Cities)


In [89]:
query7 = "SELECT s.city, FLOOR(AVG(t.duration) / 3600) AS average_hours, FLOOR((AVG(t.duration) % 3600) / 60) AS average_minutes FROM trip t JOIN station s ON t.start_station_id = s.id GROUP BY s.city;"
df7 = pd.read_sql(query7, conn)
df7.set_index("city", inplace=True)
df7.index.name = "city"
df7

,average_hours,average_minutes
city,,
Mountain View,0.0,23.0
Palo Alto,0.0,57.0
Redwood City,1.0,12.0
San Francisco,0.0,16.0
San Jose,0.0,18.0


## Rental Behavior
### Rental Behavior Difference by Weekday/Weekend

In [90]:
query8 = "SELECT EXTRACT(DOW FROM TO_TIMESTAMP(start_date, 'MM/DD/YYYY HH24:MI')) AS weekday, COUNT(*) FROM trip GROUP BY weekday;"
df8 = pd.read_sql(query8, conn)

days_of_week = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
df8["weekday_name"] = df8["weekday"].apply(lambda x: days_of_week[int(x)])
df8.drop(columns=["weekday"], inplace=True)
df8 = df8.sort_values(by="count")
df8.set_index("weekday_name", inplace=True)
df8.index.name = "weekday" 


df8

,count
weekday,
Saturday,12413
Sunday,16411
Friday,36253
Wednesday,44035
Thursday,70609
Monday,75710
Tuesday,81358


### Rental Behavior Variations by Hour of the Day

In [91]:
query9 = "SELECT EXTRACT(HOUR FROM TO_TIMESTAMP(start_date, 'MM/DD/YYYY HH24:MI')) AS hour, COUNT(*) FROM trip GROUP BY hour;"
df9 = pd.read_sql(query9, conn)
df9.set_index("hour", inplace=True)
df9.index.name = "Hour"  
df9 = df9.sort_index()

df9

,count
Hour,
0.0,1192
1.0,485
2.0,291
3.0,214
4.0,509
5.0,2621
6.0,8099
7.0,21162
8.0,42551


## Customer Segmentation
### Rental Behavior Difference between Subscribers and Casual Customers

In [92]:
query10 = "SELECT subscription_type, COUNT(*) FROM trip GROUP BY subscription_type;"
df10 = pd.read_sql(query10, conn)
df10.set_index("subscription_type", inplace=True)
df10.index.name = "subscription_type"  
df10

,count
subscription_type,
Customer,44899
Subscriber,291890


## Total Trips

In [93]:
query11 = "SELECT COUNT(*) FROM trip;"
df11 = pd.read_sql(query11, conn)
df11.set_index("count", inplace=True)
df11.index.name = "count"
df11

""
count
336789


## Close Connection

In [94]:
conn.close()

In [7]:
import folium

# Koordinaten der Stationen
station_coords = {
    70: [37.776617, -122.395260],
    50: [37.795392, -122.394203],
    77: [37.789625, -122.400811],
    55: [37.789756, -122.394643],
    60: [37.804770, -122.403234],
    61: [37.780526, -122.390288]
}

# Namen der Stationen
station_names = {
    70: "San Francisco Caltrain (Townsend at 4th)",
    50: "Harry Bridges Plaza (Ferry Building)",
    77: "Market at Sansome",
    55: "Temporary Transbay Terminal (Howard at Beale)",
    60: "Embarcadero at Sansome",
    61: "2nd at Townsend"
}

# Anzahlen der Starts und Ziele
start_counts = {70: 28338, 50: 16849, 77: 14953, 55: 14207, 60: 13504}
end_counts = {70: 32336, 77: 16837, 50: 15822, 60: 15072, 61: 12981}

# Erstellen einer Grundkarte
m = folium.Map(location=[37.7749, -122.4194], zoom_start=13)

# Hinzufügen der Marker für die Stationen
for station_id in set(start_counts) | set(end_counts):
    # Erstellen des Popup-HTML-Textes mit CSS für Schriftart und -größe
    popup_html = f"""
    <div style="font-family: Arial, sans-serif; font-size: 14px; width:200px; line-height: 1.5;">
        <strong>ID:</strong> {station_id}<br>
        <strong>Name:</strong> {station_names.get(station_id, 'N/A')}<br>
        <strong>Anzahl Start:</strong> {start_counts.get(station_id, 'N/A')}<br>
        <strong>Anzahl Ziel:</strong> {end_counts.get(station_id, 'N/A')}
    </div>
    """
    iframe = folium.IFrame(html=popup_html, width=250, height=120)
    popup = folium.Popup(iframe, max_width=265)
    marker_color = 'green' if station_id in start_counts and station_id in end_counts else 'blue' if station_id in start_counts else 'red'
    folium.Marker(
        location=station_coords[station_id],
        popup=popup,
        icon=folium.Icon(color=marker_color)
    ).add_to(m)

# Speichern der Karte als HTML-Datei
m.save('sf_bike_stations_map.html')


In [ ]:
import pandas as pd
import folium

stations_data2 = {
    2: {'lat': 37.329732, 'long': -121.901782, 'name': "San Jose Diridon Caltrain Station"},
    3: {'lat': 37.330698, 'long': -121.888979, 'name': "San Jose Civic Center"},
    4: {'lat': 37.333988, 'long': -121.894902, 'name': "Santa Clara at Almaden"},
    5: {'lat': 37.331415, 'long': -121.8932, 'name': "Adobe on Almaden"},
    6: {'lat': 37.336721, 'long': -121.894074, 'name': "San Pedro Square"},
    7: {'lat': 37.333798, 'long': -121.886943, 'name': "Paseo de San Antonio"},
    8: {'lat': 37.330165, 'long': -121.885831, 'name': "San Salvador at 1st"},
    9: {'lat': 37.348742, 'long': -121.894715, 'name': "Japantown"},
    10: {'lat': 37.337391, 'long': -121.886995, 'name': "San Jose City Hall"},
    11: {'lat': 37.335885, 'long': -121.88566000000002, 'name': "MLK Library"},
    12: {'lat': 37.332808, 'long': -121.883891, 'name': "SJSU 4th at San Carlos"},
    13: {'lat': 37.339301, 'long': -121.88993700000002, 'name': "St James Park"},
    14: {'lat': 37.332692, 'long': -121.900084, 'name': "Arena Green / SAP Center"},
    16: {'lat': 37.333955, 'long': -121.877349, 'name': "SJSU - San Salvador at 9th"},
    21: {'lat': 37.481758, 'long': -122.226904, 'name': "Franklin at Maple"},
    22: {'lat': 37.486078000000006, 'long': -122.232089, 'name': "Redwood City Caltrain Station"},
    23: {'lat': 37.487616, 'long': -122.229951, 'name': "San Mateo County Center"},
    24: {'lat': 37.484219, 'long': -122.227424, 'name': "Redwood City Public Library"},
    25: {'lat': 37.48537, 'long': -122.203288, 'name': "Stanford in Redwood City"},
    26: {'lat': 37.487682, 'long': -122.223492, 'name': "Redwood City Medical Center"},
    27: {'lat': 37.389218, 'long': -122.081896, 'name': "Mountain View City Hall"},
    28: {'lat': 37.394358, 'long': -122.07671299999998, 'name': "Mountain View Caltrain Station"},
    29: {'lat': 37.406940000000006, 'long': -122.106758, 'name': "San Antonio Caltrain Station"},
    30: {'lat': 37.390277000000005, 'long': -122.066553, 'name': "Evelyn Park and Ride"},
    31: {'lat': 37.400443, 'long': -122.108338, 'name': "San Antonio Shopping Center"},
    32: {'lat': 37.385956, 'long': -122.083678, 'name': "Castro Street and El Camino Real"},
    33: {'lat': 37.400241, 'long': -122.099076, 'name': "Rengstorff Avenue / California Street"},
    34: {'lat': 37.443988, 'long': -122.164759, 'name': "Palo Alto Caltrain Station"},
    35: {'lat': 37.444521, 'long': -122.163093, 'name': "University and Emerson"},
    36: {'lat': 37.429082, 'long': -122.142805, 'name': "California Ave Caltrain Station"},
    37: {'lat': 37.448598, 'long': -122.159504, 'name': "Cowper at University"},
    38: {'lat': 37.4256839, 'long': -122.13777749999998, 'name': "Park at Olive"},
    41: {'lat': 37.795001, 'long': -122.39997, 'name': "Clay at Battery"},
    42: {'lat': 37.79728, 'long': -122.398436, 'name': "Davis at Jackson"},
    45: {'lat': 37.794231, 'long': -122.402923, 'name': "Commercial at Montgomery"},
    46: {'lat': 37.795425, 'long': -122.404767, 'name': "Washington at Kearney"},
    47: {'lat': 37.788975, 'long': -122.403452, 'name': "Post at Kearney"},
    48: {'lat': 37.799953, 'long': -122.398525, 'name': "Embarcadero at Vallejo"},
    49: {'lat': 37.790302, 'long': -122.390637, 'name': "Spear at Folsom"},
    50: {'lat': 37.795392, 'long': -122.394203, 'name': "Harry Bridges Plaza (Ferry Building)"},
    51: {'lat': 37.791464000000005, 'long': -122.391034, 'name': "Embarcadero at Folsom"},
    39: {'lat': 37.783871000000005, 'long': -122.408433, 'name': "Powell Street BART"},
    54: {'lat': 37.787152, 'long': -122.388013, 'name': "Embarcadero at Bryant"},
    55: {'lat': 37.789756, 'long': -122.394643, 'name': "Temporary Transbay Terminal (Howard at Beale)"},
    56: {'lat': 37.792251, 'long': -122.39708600000002, 'name': "Beale at Market"},
    57: {'lat': 37.781752, 'long': -122.405127, 'name': "5th at Howard"},
    58: {'lat': 37.77865, 'long': -122.418235, 'name': "San Francisco City Hall"},
    59: {'lat': 37.781332, 'long': -122.418603, 'name': "Golden Gate at Polk"},
    60: {'lat': 37.80477, 'long': -122.403234, 'name': "Embarcadero at Sansome"},
    61: {'lat': 37.780526, 'long': -122.39028799999998, 'name': "2nd at Townsend"},
    62: {'lat': 37.785299, 'long': -122.39623600000002, 'name': "2nd at Folsom"},
    63: {'lat': 37.786978000000005, 'long': -122.398108, 'name': "Howard at 2nd"},
    64: {'lat': 37.782259, 'long': -122.392738, 'name': "2nd at South Park"},
    65: {'lat': 37.771058, 'long': -122.402717, 'name': "Townsend at 7th"},
    66: {'lat': 37.774814, 'long': -122.418954, 'name': "South Van Ness at Market"},
    67: {'lat': 37.776619, 'long': -122.417385, 'name': "Market at 10th"},
    68: {'lat': 37.784878000000006, 'long': -122.401014, 'name': "Yerba Buena Center of the Arts (3rd @ Howard)"},
    69: {'lat': 37.7766, 'long': -122.39547, 'name': "San Francisco Caltrain 2 (330 Townsend)"},
    70: {'lat': 37.776617, 'long': -122.39526, 'name': "San Francisco Caltrain (Townsend at 4th)"},
    71: {'lat': 37.788446, 'long': -122.408499, 'name': "Powell at Post (Union Square)"},
    72: {'lat': 37.781039, 'long': -122.411748, 'name': "Civic Center BART (7th at Market)"},
    73: {'lat': 37.798522, 'long': -122.407245, 'name': "Grant Avenue at Columbus Avenue"},
    74: {'lat': 37.794139, 'long': -122.394434, 'name': "Steuart at Market"},
    75: {'lat': 37.7913, 'long': -122.399051, 'name': "Mechanics Plaza (Market at Battery)"},
    76: {'lat': 37.786305, 'long': -122.404966, 'name': "Market at 4th"},
    77: {'lat': 37.789625, 'long': -122.400811, 'name': "Market at Sansome"},
    80: {'lat': 37.352601, 'long': -121.905733, 'name': "Santa Clara County Civic Center"},
    82: {'lat': 37.798541, 'long': -122.40086200000002, 'name': "Broadway St at Battery St"},
    83: {'lat': 37.491269, 'long': -122.236234, 'name': "Mezes Park"},
    84: {'lat': 37.342725, 'long': -121.895617, 'name': "Ryland Park"}
}

start_counts2 = 